In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np
from numpy.linalg import inv
import ipyvolume as ipv
import math
import re
import random
import pdb

# New Tactic

In [2]:
color_map = {
    'r':'red',
    'b':'blue',
    'g':'green',
    'w':'black',
    'y':'yellow',
    'o':'orange',
}

cube_match_res = {
    'x':re.compile('p..'),
    'y':re.compile('.n.'),
    'z':re.compile('..n'),
}

rotation_matrices={
    
    '+x':[
        [+1, +0 ,+0],
        [+0, +0, -1],
        [+0, +1, +0],
    ],
    
    '-x':[
        [+1, +0 ,+0],
        [+0, +0, +1],
        [+0, -1, +0],
    ],
    
    '+y':[
        [+0, +0, +1],
        [+0, +1, +0],
        [-1, +0, +0],
    ],
    
    '-y':[
        [+0, +0, -1],
        [+0, +1, +0],
        [+1, +0, +0],
    ],
    
    '+z':[
        [+0, -1, +0],
        [+1, +0, +0],
        [+0, +0, +1],
    ],
    
    '-z':[
        [+0, +1, +0],
        [-1, +0, +0],
        [+0, +0, +1],
    ],
    
    '+i':[
        [+1, +0, +0],
        [+0, +1, +0],
        [+0, +0, +1],
    ],
}


In [3]:
def reset():
    
    cubes = []

    diagonals = []

    for x_dir in [-1, +1]:

        for y_dir in [-1, +1]:

            for z_dir in [-1, +1]:

                diagonals += [[x_dir, y_dir, z_dir]]

    for diagonal in diagonals:

        cubes += [{
            'vectors':[
                [diagonal[0], +0, +0],
                [+0, diagonal[1], +0],
                [+0, +0, diagonal[2]],
            ],
            'colors':[
                'g' if diagonal[0] > 0 else 'b',
                'y' if diagonal[1] > 0 else 'w',
                'r' if diagonal[2] > 0 else 'o',
            ],
            'cum_rotation_matrix':[
                [1, 0, 0],
                [0, 1, 0],
                [0, 0, 1],
            ],
            'rotation_sequence':[],
        }]
        
    return cubes



In [4]:
cubes = reset()

In [5]:
def char_to_num(x):
    
    spread = 0.2
    
    if x == 'p':
    
        return +1*spread
    
    elif x == 'n':
    
        return -1*spread
    
    else:
        
        raise Exception('Incorrect string')
        
x = list(map(char_to_num, 'pnp'))

In [6]:
def num_to_char(x):
    
    if x > 0:
    
        return 'p'
    
    elif x < 0:
    
        return 'n'
    
    else:
        
        raise Exception('Incorrect num')

In [7]:
def get_quadrant(cube):

    sum_vector = np.sum(cube['vectors'], axis=0)
    
    return list(map(num_to_char, sum_vector))

In [8]:
def show_cube(*args, cubes, **kwargs):

    ipv.clear()

    ipv.pylab.xyzlim(-2,+2)
    
    for cube in cubes:

        quadrant = get_quadrant(cube)

        [x_beg, y_beg, z_beg] = list(map(char_to_num, quadrant))

        vectors  =cube['vectors']

        colors = cube['colors']

        for [x_dir, y_dir, z_dir], color in zip(vectors, colors):

#             ipv.pylab.plot(
#                 np.array([x_beg, x_beg + x_dir]), 
#                 np.array([y_beg, y_beg + y_dir]), 
#                 np.array([z_beg, z_beg + z_dir]),
#                 color=color_map[color],
#             )


            ipv.pylab.plot(
                np.array([x_beg*1.5, x_beg*1.5 + x_dir]), 
                np.array([y_beg*1.5, y_beg*1.5 + y_dir]), 
                np.array([z_beg*1.5, z_beg*1.5 + z_dir]),
                color=color_map[color],
            )
    
            ipv.pylab.scatter(
                np.array([x_beg*1.5 + x_dir]), 
                np.array([y_beg*1.5 + y_dir]), 
                np.array([z_beg*1.5 + z_dir]),
                color=color_map[color],
                marker='sphere',
                size=1,
            )

    ipv.view(20, 80)

    ipv.show()

In [9]:
def rotate_by_axis(*args, cubes, dimension, num_of_rotations, **kwargs):
    
    if num_of_rotations == 0:
        
        return cubes
        
    num_of_rotations = num_of_rotations % 4
    
    match_index_list=[]
    
    unmatched_index_list = []
    
    for index,cube in enumerate(cubes):
        
        quadrant_str = ''.join(get_quadrant(cube))
                               
        if cube_match_res[dimension[1]].match(quadrant_str):
                               
            match_index_list += [index]
            
        else:
            
            unmatched_index_list += [index]
                               
    if len(match_index_list) != 4:
                               
        raise Exception('get_positive_vectors did not find 4 vectors')
                         
    for index in match_index_list:        
                
        cubes[index]['vectors'] = np.matmul(
            np.array(rotation_matrices[dimension]), 
            np.array(cubes[index]['vectors']).T).T.tolist()     
        
        cubes[index]['cum_rotation_matrix'] = np.matmul(
            np.array(rotation_matrices[dimension]), 
            np.array(cubes[index]['cum_rotation_matrix']))
        
        cubes[index]['rotation_sequence'] += [dimension]
        
        
    for index in unmatched_index_list:        
        
        cubes[index]['rotation_sequence'] += ['+i']
        
    
    cubes = rotate_by_axis(cubes=cubes, dimension=dimension, num_of_rotations = num_of_rotations - 1)
    
    return cubes
                                               

In [10]:
cubes = reset()

original_cubes = reset()

In [11]:
show_cube(cubes=cubes); pd.DataFrame(cubes)

VBox(children=(Figure(anglex=1.3962634015954636, angley=0.3490658503988659, camera_center=[0.0, 0.0, 0.0], height=500, matrix_projection=[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], matrix_world=[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], scatters=[Scatter(color=array('blue', dtype='<U4'), color_selected=None, connected=True, size=array(1), size_selected=array(1), visible_lines=True, visible_markers=False, x=array([-0.3, -1.3]), y=array([-0.3, -0.3]), z=array([-0.3, -0.3])), Scatter(color=array('blue', dtype='<U4'), color_selected=array('white', dtype='<U5'), geo='sphere', size=array(1), size_selected=array(2.6), x=array([-1.3]), y=array([-0.3]), z=array([-0.3])), Scatter(color=array('black', dtype='<U5'), color_selected=None, connected=True, size=array(1), size_selected=array(1), visible_lines=True, visible_markers=False, x=array([-0.3, -0.3]), y=array([-0.3, -1.3]), z=array([-0.3, -0.3])), Scatter(color=array('black', dtype='<U5'), color_selected=array('white', dtype='<U5'), geo='sphere', size=array(1), size_selected=array(2.6), x=array([-0.3]), y=array([-1.3]), z=array([-0.3])), Scatter(color=array('orange', dtype='<U6'), color_selected=None, connected=True, size=array(1), size_selected=array(1), visible_lines=True, visible_markers=False, x=array([-0.3, -0.3]), y=array([-0.3, -0.3]), z=array([-0.3, -1.3])), Scatter(color=array('orange', dtype='<U6'), color_selected=array('white', dtype='<U5'), geo='sphere', size=array(1), size_selected=array(2.6), x=array([-0.3]), y=array([-0.3]), z=array([-1.3])), Scatter(color=array('blue', dtype='<U4'), color_selected=None, connected=True, size=array(1), size_selected=array(1), visible_lines=True, visible_markers=False, x=array([-0.3, -1.3]), y=array([-0.3, -0.3]), z=array([0.3, 0.3])), Scatter(color=array('blue', dtype='<U4'), color_selected=array('white', dtype='<U5'), geo='sphere', size=array(1), size_selected=array(2.6), x=array([-1.3]), y=array([-0.3]), z=array([0.3])), Scatter(color=array('black', dtype='<U5'), color_selected=None, connected=True, size=array(1), size_selected=array(1), visible_lines=True, visible_markers=False, x=array([-0.3, -0.3]), y=array([-0.3, -1.3]), z=array([0.3, 0.3])), Scatter(color=array('black', dtype='<U5'), color_selected=array('white', dtype='<U5'), geo='sphere', size=array(1), size_selected=array(2.6), x=array([-0.3]), y=array([-1.3]), z=array([0.3])), Scatter(color_selected=None, connected=True, size=array(1), size_selected=array(1), visible_lines=True, visible_markers=False, x=array([-0.3, -0.3]), y=array([-0.3, -0.3]), z=array([0.3, 1.3])), Scatter(color_selected=array('white', dtype='<U5'), geo='sphere', size=array(1), size_selected=array(2.6), x=array([-0.3]), y=array([-0.3]), z=array([1.3])), Scatter(color=array('blue', dtype='<U4'), color_selected=None, connected=True, size=array(1), size_selected=array(1), visible_lines=True, visible_markers=False, x=array([-0.3, -1.3]), y=array([0.3, 0.3]), z=array([-0.3, -0.3])), Scatter(color=array('blue', dtype='<U4'), color_selected=array('white', dtype='<U5'), geo='sphere', size=array(1), size_selected=array(2.6), x=array([-1.3]), y=array([0.3]), z=array([-0.3])), Scatter(color=array('yellow', dtype='<U6'), color_selected=None, connected=True, size=array(1), size_selected=array(1), visible_lines=True, visible_markers=False, x=array([-0.3, -0.3]), y=array([0.3, 1.3]), z=array([-0.3, -0.3])), Scatter(color=array('yellow', dtype='<U6'), color_selected=array('white', dtype='<U5'), geo='sphere', size=array(1), size_selected=array(2.6), x=array([-0.3]), y=array([1.3]), z=array([-0.3])), Scatter(color=array('orange', dtype='<U6'), color_selected=None, connected=True, size=array(1), size_selected=array(1), visible_lines=True, visible_markers=False, x=array([-0.3, -0.3]), y=array([0.3, 0.3]), z=array([-0.3, -1.3])), Scatter(color=array('orange', dtype='<U6'), color_selected=array('white', dtype='<U5'), geo='sphere', size=array(1), size_se

,colors,cum_rotation_matrix,rotation_sequence,vectors
0,"[b, w, o]","[[1, 0, 0], [0, 1, 0], [0, 0, 1]]",[],"[[-1, 0, 0], [0, -1, 0], [0, 0, -1]]"
1,"[b, w, r]","[[1, 0, 0], [0, 1, 0], [0, 0, 1]]",[],"[[-1, 0, 0], [0, -1, 0], [0, 0, 1]]"
2,"[b, y, o]","[[1, 0, 0], [0, 1, 0], [0, 0, 1]]",[],"[[-1, 0, 0], [0, 1, 0], [0, 0, -1]]"
3,"[b, y, r]","[[1, 0, 0], [0, 1, 0], [0, 0, 1]]",[],"[[-1, 0, 0], [0, 1, 0], [0, 0, 1]]"
4,"[g, w, o]","[[1, 0, 0], [0, 1, 0], [0, 0, 1]]",[],"[[1, 0, 0], [0, -1, 0], [0, 0, -1]]"
5,"[g, w, r]","[[1, 0, 0], [0, 1, 0], [0, 0, 1]]",[],"[[1, 0, 0], [0, -1, 0], [0, 0, 1]]"
6,"[g, y, o]","[[1, 0, 0], [0, 1, 0], [0, 0, 1]]",[],"[[1, 0, 0], [0, 1, 0], [0, 0, -1]]"
7,"[g, y, r]","[[1, 0, 0], [0, 1, 0], [0, 0, 1]]",[],"[[1, 0, 0], [0, 1, 0], [0, 0, 1]]"


In [12]:
#rotations = list(map(lambda x: ['x', 'y', 'z'][x], np.random.randint(low=0, high=3, size=10).tolist()))

rotations = ['+z']

In [13]:
for rotation in rotations:
    
    rotated_cubes = rotate_by_axis(
        cubes=cubes,
        dimension=rotation, 
        num_of_rotations=1
    )      

In [14]:
show_cube(cubes=cubes); pd.DataFrame(cubes)

VBox(children=(Figure(anglex=1.3962634015954636, angley=0.3490658503988659, camera_center=[0.0, 0.0, 0.0], height=500, matrix_projection=[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], matrix_world=[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], scatters=[Scatter(color=array('blue', dtype='<U4'), color_selected=None, connected=True, size=array(1), size_selected=array(1), visible_lines=True, visible_markers=False, x=array([0.3, 0.3]), y=array([-0.3, -1.3]), z=array([-0.3, -0.3])), Scatter(color=array('blue', dtype='<U4'), color_selected=array('white', dtype='<U5'), geo='sphere', size=array(1), size_selected=array(2.6), x=array([0.3]), y=array([-1.3]), z=array([-0.3])), Scatter(color=array('black', dtype='<U5'), color_selected=None, connected=True, size=array(1), size_selected=array(1), visible_lines=True, visible_markers=False, x=array([0.3, 1.3]), y=array([-0.3, -0.3]), z=array([-0.3, -0.3])), Scatter(color=array('black', dtype='<U5'), color_selected=array('white', dtype='<U5'), geo='sphere', size=array(1), size_selected=array(2.6), x=array([1.3]), y=array([-0.3]), z=array([-0.3])), Scatter(color=array('orange', dtype='<U6'), color_selected=None, connected=True, size=array(1), size_selected=array(1), visible_lines=True, visible_markers=False, x=array([0.3, 0.3]), y=array([-0.3, -0.3]), z=array([-0.3, -1.3])), Scatter(color=array('orange', dtype='<U6'), color_selected=array('white', dtype='<U5'), geo='sphere', size=array(1), size_selected=array(2.6), x=array([0.3]), y=array([-0.3]), z=array([-1.3])), Scatter(color=array('blue', dtype='<U4'), color_selected=None, connected=True, size=array(1), size_selected=array(1), visible_lines=True, visible_markers=False, x=array([-0.3, -1.3]), y=array([-0.3, -0.3]), z=array([0.3, 0.3])), Scatter(color=array('blue', dtype='<U4'), color_selected=array('white', dtype='<U5'), geo='sphere', size=array(1), size_selected=array(2.6), x=array([-1.3]), y=array([-0.3]), z=array([0.3])), Scatter(color=array('black', dtype='<U5'), color_selected=None, connected=True, size=array(1), size_selected=array(1), visible_lines=True, visible_markers=False, x=array([-0.3, -0.3]), y=array([-0.3, -1.3]), z=array([0.3, 0.3])), Scatter(color=array('black', dtype='<U5'), color_selected=array('white', dtype='<U5'), geo='sphere', size=array(1), size_selected=array(2.6), x=array([-0.3]), y=array([-1.3]), z=array([0.3])), Scatter(color_selected=None, connected=True, size=array(1), size_selected=array(1), visible_lines=True, visible_markers=False, x=array([-0.3, -0.3]), y=array([-0.3, -0.3]), z=array([0.3, 1.3])), Scatter(color_selected=array('white', dtype='<U5'), geo='sphere', size=array(1), size_selected=array(2.6), x=array([-0.3]), y=array([-0.3]), z=array([1.3])), Scatter(color=array('blue', dtype='<U4'), color_selected=None, connected=True, size=array(1), size_selected=array(1), visible_lines=True, visible_markers=False, x=array([-0.3, -0.3]), y=array([-0.3, -1.3]), z=array([-0.3, -0.3])), Scatter(color=array('blue', dtype='<U4'), color_selected=array('white', dtype='<U5'), geo='sphere', size=array(1), size_selected=array(2.6), x=array([-0.3]), y=array([-1.3]), z=array([-0.3])), Scatter(color=array('yellow', dtype='<U6'), color_selected=None, connected=True, size=array(1), size_selected=array(1), visible_lines=True, visible_markers=False, x=array([-0.3, -1.3]), y=array([-0.3, -0.3]), z=array([-0.3, -0.3])), Scatter(color=array('yellow', dtype='<U6'), color_selected=array('white', dtype='<U5'), geo='sphere', size=array(1), size_selected=array(2.6), x=array([-1.3]), y=array([-0.3]), z=array([-0.3])), Scatter(color=array('orange', dtype='<U6'), color_selected=None, connected=True, size=array(1), size_selected=array(1), visible_lines=True, visible_markers=False, x=array([-0.3, -0.3]), y=array([-0.3, -0.3]), z=array([-0.3, -1.3])), Scatter(color=array('orange', dtype='<U6'), color_selected=array('white', dtype='<U5'), geo='sphere', size=array(1), size_sel

,colors,cum_rotation_matrix,rotation_sequence,vectors
0,"[b, w, o]","[[0, -1, 0], [1, 0, 0], [0, 0, 1]]",[+z],"[[0, -1, 0], [1, 0, 0], [0, 0, -1]]"
1,"[b, w, r]","[[1, 0, 0], [0, 1, 0], [0, 0, 1]]",[+i],"[[-1, 0, 0], [0, -1, 0], [0, 0, 1]]"
2,"[b, y, o]","[[0, -1, 0], [1, 0, 0], [0, 0, 1]]",[+z],"[[0, -1, 0], [-1, 0, 0], [0, 0, -1]]"
3,"[b, y, r]","[[1, 0, 0], [0, 1, 0], [0, 0, 1]]",[+i],"[[-1, 0, 0], [0, 1, 0], [0, 0, 1]]"
4,"[g, w, o]","[[0, -1, 0], [1, 0, 0], [0, 0, 1]]",[+z],"[[0, 1, 0], [1, 0, 0], [0, 0, -1]]"
5,"[g, w, r]","[[1, 0, 0], [0, 1, 0], [0, 0, 1]]",[+i],"[[1, 0, 0], [0, -1, 0], [0, 0, 1]]"
6,"[g, y, o]","[[0, -1, 0], [1, 0, 0], [0, 0, 1]]",[+z],"[[0, 1, 0], [-1, 0, 0], [0, 0, -1]]"
7,"[g, y, r]","[[1, 0, 0], [0, 1, 0], [0, 0, 1]]",[+i],"[[1, 0, 0], [0, 1, 0], [0, 0, 1]]"


# Examples

## starting and endings points for cube #1

In [15]:
beg = np.array([[-1, 0, 0], [0, -1, 0], [0, 0, -1]])
end = np.array([[0, 0, -1], [1, 0, 0], [0, 1, 0]])
trans_fwd = np.array([[0, -1, 0], [0, 0, -1], [1, 0, 0]])

## get forward rotation matrix using matrix math

In [16]:
np.matmul(
    end.T,
    inv(beg.T)
).astype(int)

array([[ 0, -1,  0],
       [ 0,  0, -1],
       [ 1,  0,  0]])

## check if the data from the cum_rotation_matrix worked

In [17]:
np.matmul(
    trans_fwd,
    beg.T
).T

array([[ 0,  0, -1],
       [ 1,  0,  0],
       [ 0,  1,  0]])

## get the reverse rotation matrix - going from end to the beginning

In [18]:
np.matmul(
    beg.T,
    inv(end.T)
).astype(int)

array([[ 0,  0,  1],
       [-1,  0,  0],
       [ 0, -1,  0]])

## just invert the forward rotation matrix - the answer should be the same

In [19]:
inv(trans_fwd).astype(int)

array([[ 0,  0,  1],
       [-1,  0,  0],
       [ 0, -1,  0]])

# finally use the actual rotations documented

In [20]:
trans = np.identity(3); beg

for rotation in ['-x', '-z']:

    trans = np.matmul(rotation_matrices[rotation], trans)
    
trans.astype(int)

array([[ 0,  0,  1],
       [-1,  0,  0],
       [ 0, -1,  0]])

# Use matrix math to solve rotation

In [21]:
unsolved_cubes = [
    {
        'colors':['g', 'y', 'o'],
        'vectors':[[+0, -1, +0], [+0, +0, +1], [+1, +0, +0]],
    },
    {
        'colors':['w', 'b', 'r'],
        'vectors':[[+0, -1, 0], [-1, +0, +0], [+0, +0, +1]],
    },  
    {
        'colors':['r', 'w', 'g'],
        'vectors':[[+0, -1, +0], [-1, +0, +0], [+0, +0, -1]],
    },
    {
        'colors':['o', 'b', 'w'],
        'vectors':[[+0, -1, +0], [+1, +0, +0], [+0, +0, -1]],
    },
    {
        'colors':['b', 'r', 'y'],
        'vectors':[[+0, +1, +0], [-1, +0, +0], [+0, +0, +1]],
    },
    {
        'colors':['o', 'y', 'b'],
        'vectors':[[+0, +1, +0], [+1, +0, +0], [+0, +0, +1]],
    },
    {
        'colors':['w', 'o', 'g'],
        'vectors':[[+0, +1, +0], [+1, +0, +0], [+0, +0, -1]],
    },
    {
        'colors':['g', 'r', 'y'],
        'vectors':[[+0, +1, +0], [-1, +0, +0], [+0, +0, -1]],
    },
]

for cube in unsolved_cubes:
    
    cube['cum_solution_matrix']=[
        [1, 0, 0],
        [0, 1, 0],
        [0, 0, 1], 
    ]
                
    cube['rotation_sequence'] = []


In [22]:
def get_cube(*args, cubes, colors, **kwargs):
    
    requested_cube = sorted(colors)
    
    for index, attributes in enumerate(cubes):
        
        if sorted(attributes['colors']) == requested_cube:
            
            indices = list(map(attributes['colors'].index, requested_cube))
            
            return {'colors':requested_cube, 'vectors':[attributes['vectors'][i] for i in indices]}
    

In [23]:
def get_cum_rotation_matrix(*args, beg_cube, end_cube, **kwargs):
    
    if beg_cube['colors'] != end_cube['colors']:
        
        raise Exception('Colors sequence do not match')
        
    beg = beg_cube['vectors']
    
    end = end_cube['vectors']
  
    matrix = np.matmul(
        np.array(end).T,
        inv(np.array(beg).T)).astype(int)  

    return matrix    

In [24]:
for cube in unsolved_cubes:

    beg_cube = get_cube(cubes=original_cubes, colors=cube['colors'])

    end_cube = get_cube(cubes=unsolved_cubes, colors=cube['colors'])

    cube['cum_solution_matrix'] = get_cum_rotation_matrix(
        beg_cube=end_cube, 
        end_cube=beg_cube, 
    )

In [25]:
possibilities = ['+x', '-x', '+y', '-y', '+z', '-z', '+i']

combinations = pd.MultiIndex.from_product([possibilities for _ in range(3)])

In [26]:
combinations.tolist()

[('+x', '+x', '+x'),
 ('+x', '+x', '-x'),
 ('+x', '+x', '+y'),
 ('+x', '+x', '-y'),
 ('+x', '+x', '+z'),
 ('+x', '+x', '-z'),
 ('+x', '+x', '+i'),
 ('+x', '-x', '+x'),
 ('+x', '-x', '-x'),
 ('+x', '-x', '+y'),
 ('+x', '-x', '-y'),
 ('+x', '-x', '+z'),
 ('+x', '-x', '-z'),
 ('+x', '-x', '+i'),
 ('+x', '+y', '+x'),
 ('+x', '+y', '-x'),
 ('+x', '+y', '+y'),
 ('+x', '+y', '-y'),
 ('+x', '+y', '+z'),
 ('+x', '+y', '-z'),
 ('+x', '+y', '+i'),
 ('+x', '-y', '+x'),
 ('+x', '-y', '-x'),
 ('+x', '-y', '+y'),
 ('+x', '-y', '-y'),
 ('+x', '-y', '+z'),
 ('+x', '-y', '-z'),
 ('+x', '-y', '+i'),
 ('+x', '+z', '+x'),
 ('+x', '+z', '-x'),
 ('+x', '+z', '+y'),
 ('+x', '+z', '-y'),
 ('+x', '+z', '+z'),
 ('+x', '+z', '-z'),
 ('+x', '+z', '+i'),
 ('+x', '-z', '+x'),
 ('+x', '-z', '-x'),
 ('+x', '-z', '+y'),
 ('+x', '-z', '-y'),
 ('+x', '-z', '+z'),
 ('+x', '-z', '-z'),
 ('+x', '-z', '+i'),
 ('+x', '+i', '+x'),
 ('+x', '+i', '-x'),
 ('+x', '+i', '+y'),
 ('+x', '+i', '-y'),
 ('+x', '+i', '+z'),
 ('+x', '+i',

In [27]:
cum_solution_matrices = []

for combination in combinations.tolist():

    state = np.array(rotation_matrices['+i'])

    for rotation in combination:

        state = np.matmul(
            np.array(rotation_matrices[rotation]), 
            state,
        )
        
    cum_solution_matrices += [(state, combination)]

#     if (state==cum_rotation_matrix).flatten().sum()==9:
        
#         print(combination)

In [28]:
def get_possible_rotation_sequence(*args, cum_solution_matrix, solution_matrices, **kwargs):
    
    for index, (possible_match, rotation) in enumerate(solution_matrices):
                
        if (possible_match==cum_solution_matrix).flatten().sum()==9:
            
            return rotation
        
    return None

In [29]:
unsolved_cubes

[{'colors': ['g', 'y', 'o'], 'cum_solution_matrix': array([[ 0, -1,  0],
         [ 0,  0,  1],
         [-1,  0,  0]]), 'rotation_sequence': [], 'vectors': [[0, -1, 0],
   [0, 0, 1],
   [1, 0, 0]]},
 {'colors': ['w', 'b', 'r'], 'cum_solution_matrix': array([[1, 0, 0],
         [0, 1, 0],
         [0, 0, 1]]), 'rotation_sequence': [], 'vectors': [[0, -1, 0],
   [-1, 0, 0],
   [0, 0, 1]]},
 {'colors': ['r', 'w', 'g'], 'cum_solution_matrix': array([[ 0,  0, -1],
         [ 1,  0,  0],
         [ 0, -1,  0]]), 'rotation_sequence': [], 'vectors': [[0, -1, 0],
   [-1, 0, 0],
   [0, 0, -1]]},
 {'colors': ['o', 'b', 'w'], 'cum_solution_matrix': array([[-1,  0,  0],
         [ 0,  0,  1],
         [ 0,  1,  0]]), 'rotation_sequence': [], 'vectors': [[0, -1, 0],
   [1, 0, 0],
   [0, 0, -1]]},
 {'colors': ['b', 'r', 'y'], 'cum_solution_matrix': array([[ 0, -1,  0],
         [ 0,  0,  1],
         [-1,  0,  0]]), 'rotation_sequence': [], 'vectors': [[0, 1, 0],
   [-1, 0, 0],
   [0, 0, 1]]},
 {'co

In [30]:
for cube in unsolved_cubes:

    rotation_sequence = get_possible_rotation_sequence(
        cum_solution_matrix=cube['cum_solution_matrix'],
        solution_matrices=cum_solution_matrices
    )
    
    cube['rotation_sequence'] = rotation_sequence

In [31]:
pd.DataFrame(unsolved_cubes)

,colors,cum_solution_matrix,rotation_sequence,vectors
0,"[g, y, o]","[[0, -1, 0], [0, 0, 1], [-1, 0, 0]]","(-x, +y, +i)","[[0, -1, 0], [0, 0, 1], [1, 0, 0]]"
1,"[w, b, r]","[[1, 0, 0], [0, 1, 0], [0, 0, 1]]","(+x, -x, +i)","[[0, -1, 0], [-1, 0, 0], [0, 0, 1]]"
2,"[r, w, g]","[[0, 0, -1], [1, 0, 0], [0, -1, 0]]","(-x, +z, +i)","[[0, -1, 0], [-1, 0, 0], [0, 0, -1]]"
3,"[o, b, w]","[[-1, 0, 0], [0, 0, 1], [0, 1, 0]]","(+x, +z, +z)","[[0, -1, 0], [1, 0, 0], [0, 0, -1]]"
4,"[b, r, y]","[[0, -1, 0], [0, 0, 1], [-1, 0, 0]]","(-x, +y, +i)","[[0, 1, 0], [-1, 0, 0], [0, 0, 1]]"
5,"[o, y, b]","[[0, 0, -1], [1, 0, 0], [0, -1, 0]]","(-x, +z, +i)","[[0, 1, 0], [1, 0, 0], [0, 0, 1]]"
6,"[w, o, g]","[[0, 0, -1], [0, -1, 0], [-1, 0, 0]]","(+x, +x, +y)","[[0, 1, 0], [1, 0, 0], [0, 0, -1]]"
7,"[g, r, y]","[[0, 1, 0], [0, 0, -1], [-1, 0, 0]]","(+x, +y, +i)","[[0, 1, 0], [-1, 0, 0], [0, 0, -1]]"


In [ ]:
def get_quadrant_re_for_cube_and_dimension(*args, cube, dimension, **kwargs):
    
    quadrant_str = get_quadrant(cube):

    
    

In [34]:
def rotate_by_axis(*args, cubes, target_cube_index, dimension, num_of_rotations, **kwargs):

    if num_of_rotations == 0:
        
        return cubes
        
    num_of_rotations = num_of_rotations % 4
    
    match_index_list=[]
    
    unmatched_index_list = []
    
    cube_match_re = get_quadrant_re_for_cube_and_dimension(cubes[target_cube_index], dimension)
       
    for index,cube in enumerate(cubes):
        
        quadrant_str = ''.join(get_quadrant(cube))
                               
        if cube_match_res[dimension[1]].match(quadrant_str):
                               
            match_index_list += [index]
            
        else:
            
            unmatched_index_list += [index]


VBox(children=(Figure(anglex=1.3962634015954636, angley=0.3490658503988659, camera_center=[0.0, 0.0, 0.0], height=500, matrix_projection=[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], matrix_world=[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], scatters=[Scatter(color=array('green', dtype='<U5'), color_selected=None, connected=True, size=array(1), size_selected=array(1), visible_lines=True, visible_markers=False, x=array([0.3, 0.3]), y=array([-0.3, -1.3]), z=array([0.3, 0.3])), Scatter(color=array('green', dtype='<U5'), color_selected=array('white', dtype='<U5'), geo='sphere', size=array(1), size_selected=array(2.6), x=array([0.3]), y=array([-1.3]), z=array([0.3])), Scatter(color=array('yellow', dtype='<U6'), color_selected=None, connected=True, size=array(1), size_selected=array(1), visible_lines=True, visible_markers=False, x=array([0.3, 0.3]), y=array([-0.3, -0.3]), z=array([0.3, 1.3])), Scatter(color=array('yellow', dtype='<U6'), color_selected=array('white', dtype='<U5'), geo='sphere', size=array(1), size_selected=array(2.6), x=array([0.3]), y=array([-0.3]), z=array([1.3])), Scatter(color=array('orange', dtype='<U6'), color_selected=None, connected=True, size=array(1), size_selected=array(1), visible_lines=True, visible_markers=False, x=array([0.3, 1.3]), y=array([-0.3, -0.3]), z=array([0.3, 0.3])), Scatter(color=array('orange', dtype='<U6'), color_selected=array('white', dtype='<U5'), geo='sphere', size=array(1), size_selected=array(2.6), x=array([1.3]), y=array([-0.3]), z=array([0.3])), Scatter(color=array('black', dtype='<U5'), color_selected=None, connected=True, size=array(1), size_selected=array(1), visible_lines=True, visible_markers=False, x=array([-0.3, -0.3]), y=array([-0.3, -1.3]), z=array([0.3, 0.3])), Scatter(color=array('black', dtype='<U5'), color_selected=array('white', dtype='<U5'), geo='sphere', size=array(1), size_selected=array(2.6), x=array([-0.3]), y=array([-1.3]), z=array([0.3])), Scatter(color=array('blue', dtype='<U4'), color_selected=None, connected=True, size=array(1), size_selected=array(1), visible_lines=True, visible_markers=False, x=array([-0.3, -1.3]), y=array([-0.3, -0.3]), z=array([0.3, 0.3])), Scatter(color=array('blue', dtype='<U4'), color_selected=array('white', dtype='<U5'), geo='sphere', size=array(1), size_selected=array(2.6), x=array([-1.3]), y=array([-0.3]), z=array([0.3])), Scatter(color_selected=None, connected=True, size=array(1), size_selected=array(1), visible_lines=True, visible_markers=False, x=array([-0.3, -0.3]), y=array([-0.3, -0.3]), z=array([0.3, 1.3])), Scatter(color_selected=array('white', dtype='<U5'), geo='sphere', size=array(1), size_selected=array(2.6), x=array([-0.3]), y=array([-0.3]), z=array([1.3])), Scatter(color_selected=None, connected=True, size=array(1), size_selected=array(1), visible_lines=True, visible_markers=False, x=array([-0.3, -0.3]), y=array([-0.3, -1.3]), z=array([-0.3, -0.3])), Scatter(color_selected=array('white', dtype='<U5'), geo='sphere', size=array(1), size_selected=array(2.6), x=array([-0.3]), y=array([-1.3]), z=array([-0.3])), Scatter(color=array('black', dtype='<U5'), color_selected=None, connected=True, size=array(1), size_selected=array(1), visible_lines=True, visible_markers=False, x=array([-0.3, -1.3]), y=array([-0.3, -0.3]), z=array([-0.3, -0.3])), Scatter(color=array('black', dtype='<U5'), color_selected=array('white', dtype='<U5'), geo='sphere', size=array(1), size_selected=array(2.6), x=array([-1.3]), y=array([-0.3]), z=array([-0.3])), Scatter(color=array('green', dtype='<U5'), color_selected=None, connected=True, size=array(1), size_selected=array(1), visible_lines=True, visible_markers=False, x=array([-0.3, -0.3]), y=array([-0.3, -0.3]), z=array([-0.3, -1.3])), Scatter(color=array('green', dtype='<U5'), color_selected=array('white', dtype='<U5'), geo='sphere', size=array(1), size_selected=array(2.6), x=array([-0.3]), y=array([-0.3]), z=array([-1.3])), Scatter